In [ ]:
'''
Initial code was copied from:
https://github.com/eurismarpires/Keras-GAN-1/blob/master/gan/gan.py
'''

In [25]:
'''
    Unsupervised Representation Learning with Deep Convolutional Generative Adversarial Networks
    Ref:
        - https://arxiv.org/abs/1511.06434
'''

import tensorflow as tf
import keras.backend as K
from keras.models import Sequential
from keras.layers import Conv2D,GlobalAveragePooling2D,LeakyReLU,Conv2DTranspose,Activation,BatchNormalization
from keras.optimizers import Adam

from keras.layers import Dense
from keras import initializers
from keras.layers.core import Dropout




def build_generator(input_shape):
#     model = Sequential()

#     model.add(Conv2DTranspose(512,(3,3),strides=(2,2),padding="same",input_shape=input_shape))
#     model.add(LeakyReLU(0.2))
#     model.add(BatchNormalization())

#     model.add(Conv2DTranspose(256,(3,3),strides=(2,2),padding="same"))
#     model.add(LeakyReLU(0.2))
#     model.add(BatchNormalization())

#     model.add(Conv2DTranspose(128,(3,3),strides=(2,2),padding="same"))
#     model.add(LeakyReLU(0.2))
#     model.add(BatchNormalization())

#     model.add(Conv2DTranspose(64,(3,3),strides=(2,2),padding="same"))
#     model.add(LeakyReLU(0.2))
#     model.add(BatchNormalization())

#     model.add(Conv2D(3,(3,3),padding="same",activation="tanh"))

    generator = Sequential()
    
    generator.add(Dense(512,
                        input_dim=5,#noise_dim,
                        kernel_initializer=initializers.RandomNormal(stddev=0.02)))
    generator.add(BatchNormalization())
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(512))
    generator.add(BatchNormalization())
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(512))
    generator.add(BatchNormalization())
    generator.add(LeakyReLU(0.2))
    

    
    generator.add(Dense(512))
    generator.add(BatchNormalization())
    generator.add(LeakyReLU(0.2))
    
    
    generator.add(Dense(512))
    generator.add(BatchNormalization())
    generator.add(LeakyReLU(0.2))
    
    
    generator.add(Dense(105, activation='tanh'))
    return generator


def build_discriminator(input_shape):
#     model = Sequential()

#     model.add(Conv2D(64,(3,3),strides=(2,2),padding="same",input_shape=input_shape))
#     model.add(LeakyReLU(0.2))
#     model.add(BatchNormalization())

#     model.add(Conv2D(128,(3,3),strides=(2,2),padding="same"))
#     model.add(LeakyReLU(0.2))
#     model.add(BatchNormalization())

#     model.add(Conv2D(256,(3,3),strides=(2,2),padding="same"))
#     model.add(LeakyReLU(0.2))
#     model.add(BatchNormalization())

#     model.add(Conv2D(512,(3,3),strides=(2,2),padding="same"))
#     model.add(LeakyReLU(0.2))
#     model.add(BatchNormalization())

#     model.add(Conv2D(1,(3,3),padding="same"))
#     model.add(GlobalAveragePooling2D())
#     model.add(Activation("sigmoid"))

    discriminator = Sequential()
    discriminator.add(Dense(512,
                            input_dim=105,
                            kernel_initializer=initializers.RandomNormal(stddev=0.02)))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))
    
    
    discriminator.add(Dense(512))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))
    
    discriminator.add(Dense(512))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))
        
    discriminator.add(Dense(512))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))
        
    discriminator.add(Dense(512))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))
    
    discriminator.add(Dense(1, activation='sigmoid'))

    return discriminator

def build_functions(batch_size, noise_size, image_size, generator, discriminator):

    noise = K.random_normal((batch_size,) + noise_size,0.0,1.0,"float32")
    real_image = K.placeholder((batch_size,) + image_size)
    fake_image = generator(noise)

    d_input = K.concatenate([real_image, fake_image], axis=0)
    pred_real, pred_fake = tf.split(discriminator(d_input), num_or_size_splits = 2, axis = 0)

    pred_real = K.clip(pred_real,K.epsilon(),1-K.epsilon())
    pred_fake = K.clip(pred_fake,K.epsilon(),1-K.epsilon())

    d_loss = -(K.mean(K.log(pred_real)) + K.mean(K.log(1-pred_fake)))
    g_loss = -K.mean(K.log(pred_fake))

    # get updates of mean and variance in batch normalization layers
    d_updates = discriminator.get_updates_for([d_input])
    g_updates = generator.get_updates_for([noise])

    d_training_updates = Adam(lr=0.0001, beta_1=0.0, beta_2=0.9).get_updates(d_loss, discriminator.trainable_weights)
    d_train = K.function([real_image, K.learning_phase()], [d_loss],d_updates + d_training_updates)

    g_training_updates = Adam(lr=0.0001, beta_1=0.0, beta_2=0.9).get_updates(g_loss, generator.trainable_weights)
    g_train = K.function([real_image, K.learning_phase()], [g_loss], g_updates + g_training_updates)

    return d_train,g_train

In [10]:
# Loading MHAD data for action1, all persons and all repeatations of each person
from utils.data_loader import data_loader
data_object= data_loader(matlab_action_path='../../')
myData, mymin, mymax = data_object.actions_normalised([1], twoD_true_or_threeD_false=False)
myData.shape

(117562, 105)

In [22]:
import keras

#from gan_libs.DCGAN import build_generator, build_discriminator, build_functions
# from gan_libs.LSGAN import build_generator, build_discriminator, build_functions
# from gan_libs.SNGAN import build_generator, build_discriminator, build_functions
# from gan_libs.WGAN_GP import build_generator, build_discriminator, build_functions

from utils.common import set_gpu_config, predict_images
import numpy as np

set_gpu_config("0",0.5)

epoch = 500
steps = 1000
image_size = (1,1,105)
noise_size = (1,1,5)
batch_size = 16

x_train = myData

generator = build_generator(noise_size)
discriminator = build_discriminator(image_size)
d_train, g_train = build_functions(batch_size, noise_size, image_size, generator, discriminator)

for e in range(epoch):
    for s in range(steps):
        real_images = x_train[np.random.permutation(x_train.shape[0])[:batch_size]]
        real_images.shape = (batch_size,1,1,105)
        print(real_images.shape)
        d_loss, = d_train([real_images, 1])
        g_loss, = g_train([real_images, 1])
        print ("[{0}/{1}] [{2}/{3}] d_loss: {4:.4}, g_loss: {5:.4}".format(e, epoch, s, steps, d_loss, g_loss))
    if e % 100 == 0:
        generator.save_weights("e{0}_generator.h5".format(e))
        discriminator.save_weights("e{0}_discriminator.h5".format(e))
    #predict_images("e{0}_img.png".format(e), generator,noise_size,10,32)


(16, 1, 1, 105)
[0/50] [0/1000] d_loss: 1.421, g_loss: 0.7767
(16, 1, 1, 105)
[0/50] [1/1000] d_loss: 1.346, g_loss: 0.8034
(16, 1, 1, 105)
[0/50] [2/1000] d_loss: 1.338, g_loss: 0.7889
(16, 1, 1, 105)
[0/50] [3/1000] d_loss: 1.272, g_loss: 0.7419
(16, 1, 1, 105)
[0/50] [4/1000] d_loss: 1.309, g_loss: 0.8266
(16, 1, 1, 105)
[0/50] [5/1000] d_loss: 1.288, g_loss: 0.802
(16, 1, 1, 105)
[0/50] [6/1000] d_loss: 1.254, g_loss: 0.8231
(16, 1, 1, 105)
[0/50] [7/1000] d_loss: 1.228, g_loss: 0.8262
(16, 1, 1, 105)
[0/50] [8/1000] d_loss: 1.237, g_loss: 0.7953
(16, 1, 1, 105)
[0/50] [9/1000] d_loss: 1.2, g_loss: 0.7981
(16, 1, 1, 105)
[0/50] [10/1000] d_loss: 1.166, g_loss: 0.8188
(16, 1, 1, 105)
[0/50] [11/1000] d_loss: 1.149, g_loss: 0.9073
(16, 1, 1, 105)
[0/50] [12/1000] d_loss: 1.152, g_loss: 0.8693
(16, 1, 1, 105)
[0/50] [13/1000] d_loss: 1.136, g_loss: 0.9568
(16, 1, 1, 105)
[0/50] [14/1000] d_loss: 1.074, g_loss: 0.8447
(16, 1, 1, 105)
[0/50] [15/1000] d_loss: 1.148, g_loss: 0.931
(16, 1

[0/50] [134/1000] d_loss: 0.7809, g_loss: 1.303
(16, 1, 1, 105)
[0/50] [135/1000] d_loss: 0.9287, g_loss: 1.344
(16, 1, 1, 105)
[0/50] [136/1000] d_loss: 0.8835, g_loss: 1.873
(16, 1, 1, 105)
[0/50] [137/1000] d_loss: 0.986, g_loss: 1.536
(16, 1, 1, 105)
[0/50] [138/1000] d_loss: 0.986, g_loss: 1.505
(16, 1, 1, 105)
[0/50] [139/1000] d_loss: 0.8789, g_loss: 1.329
(16, 1, 1, 105)
[0/50] [140/1000] d_loss: 1.073, g_loss: 1.834
(16, 1, 1, 105)
[0/50] [141/1000] d_loss: 0.8448, g_loss: 1.341
(16, 1, 1, 105)
[0/50] [142/1000] d_loss: 0.8514, g_loss: 1.604
(16, 1, 1, 105)
[0/50] [143/1000] d_loss: 0.915, g_loss: 1.729
(16, 1, 1, 105)
[0/50] [144/1000] d_loss: 0.8707, g_loss: 1.592
(16, 1, 1, 105)
[0/50] [145/1000] d_loss: 0.7378, g_loss: 1.582
(16, 1, 1, 105)
[0/50] [146/1000] d_loss: 0.8392, g_loss: 1.419
(16, 1, 1, 105)
[0/50] [147/1000] d_loss: 0.9136, g_loss: 1.772
(16, 1, 1, 105)
[0/50] [148/1000] d_loss: 0.9298, g_loss: 1.981
(16, 1, 1, 105)
[0/50] [149/1000] d_loss: 0.8702, g_loss: 1.

[0/50] [269/1000] d_loss: 0.5213, g_loss: 2.331
(16, 1, 1, 105)
[0/50] [270/1000] d_loss: 0.5942, g_loss: 2.883
(16, 1, 1, 105)
[0/50] [271/1000] d_loss: 0.5433, g_loss: 2.235
(16, 1, 1, 105)
[0/50] [272/1000] d_loss: 0.5139, g_loss: 3.427
(16, 1, 1, 105)
[0/50] [273/1000] d_loss: 0.4318, g_loss: 1.803
(16, 1, 1, 105)
[0/50] [274/1000] d_loss: 0.5471, g_loss: 2.83
(16, 1, 1, 105)
[0/50] [275/1000] d_loss: 0.5683, g_loss: 1.898
(16, 1, 1, 105)
[0/50] [276/1000] d_loss: 0.4531, g_loss: 2.31
(16, 1, 1, 105)
[0/50] [277/1000] d_loss: 0.4595, g_loss: 2.539
(16, 1, 1, 105)
[0/50] [278/1000] d_loss: 0.5148, g_loss: 2.588
(16, 1, 1, 105)
[0/50] [279/1000] d_loss: 0.377, g_loss: 1.581
(16, 1, 1, 105)
[0/50] [280/1000] d_loss: 0.6093, g_loss: 4.263
(16, 1, 1, 105)
[0/50] [281/1000] d_loss: 0.6596, g_loss: 1.652
(16, 1, 1, 105)
[0/50] [282/1000] d_loss: 0.5134, g_loss: 3.045
(16, 1, 1, 105)
[0/50] [283/1000] d_loss: 0.4748, g_loss: 2.342
(16, 1, 1, 105)
[0/50] [284/1000] d_loss: 0.3851, g_loss: 2

[0/50] [404/1000] d_loss: 0.2269, g_loss: 2.178
(16, 1, 1, 105)
[0/50] [405/1000] d_loss: 0.3483, g_loss: 3.911
(16, 1, 1, 105)
[0/50] [406/1000] d_loss: 0.3125, g_loss: 3.752
(16, 1, 1, 105)
[0/50] [407/1000] d_loss: 0.3231, g_loss: 3.155
(16, 1, 1, 105)
[0/50] [408/1000] d_loss: 0.3623, g_loss: 3.334
(16, 1, 1, 105)
[0/50] [409/1000] d_loss: 0.2564, g_loss: 3.244
(16, 1, 1, 105)
[0/50] [410/1000] d_loss: 0.2714, g_loss: 2.595
(16, 1, 1, 105)
[0/50] [411/1000] d_loss: 0.3428, g_loss: 3.708
(16, 1, 1, 105)
[0/50] [412/1000] d_loss: 0.3157, g_loss: 2.957
(16, 1, 1, 105)
[0/50] [413/1000] d_loss: 0.3369, g_loss: 2.919
(16, 1, 1, 105)
[0/50] [414/1000] d_loss: 0.283, g_loss: 2.984
(16, 1, 1, 105)
[0/50] [415/1000] d_loss: 0.2441, g_loss: 2.826
(16, 1, 1, 105)
[0/50] [416/1000] d_loss: 0.2865, g_loss: 3.036
(16, 1, 1, 105)
[0/50] [417/1000] d_loss: 0.4274, g_loss: 4.676
(16, 1, 1, 105)
[0/50] [418/1000] d_loss: 0.4108, g_loss: 3.029
(16, 1, 1, 105)
[0/50] [419/1000] d_loss: 0.2656, g_loss:

(16, 1, 1, 105)
[0/50] [534/1000] d_loss: 0.7024, g_loss: 1.631
(16, 1, 1, 105)
[0/50] [535/1000] d_loss: 0.8352, g_loss: 4.248
(16, 1, 1, 105)
[0/50] [536/1000] d_loss: 0.313, g_loss: 2.719
(16, 1, 1, 105)
[0/50] [537/1000] d_loss: 0.2162, g_loss: 3.194
(16, 1, 1, 105)
[0/50] [538/1000] d_loss: 0.2778, g_loss: 4.418
(16, 1, 1, 105)
[0/50] [539/1000] d_loss: 0.4338, g_loss: 2.619
(16, 1, 1, 105)
[0/50] [540/1000] d_loss: 0.2116, g_loss: 2.093
(16, 1, 1, 105)
[0/50] [541/1000] d_loss: 0.2828, g_loss: 3.001
(16, 1, 1, 105)
[0/50] [542/1000] d_loss: 0.3325, g_loss: 3.051
(16, 1, 1, 105)
[0/50] [543/1000] d_loss: 0.2752, g_loss: 3.47
(16, 1, 1, 105)
[0/50] [544/1000] d_loss: 0.3434, g_loss: 3.768
(16, 1, 1, 105)
[0/50] [545/1000] d_loss: 0.1393, g_loss: 3.098
(16, 1, 1, 105)
[0/50] [546/1000] d_loss: 0.411, g_loss: 4.325
(16, 1, 1, 105)
[0/50] [547/1000] d_loss: 0.3854, g_loss: 2.525
(16, 1, 1, 105)
[0/50] [548/1000] d_loss: 0.4673, g_loss: 3.312
(16, 1, 1, 105)
[0/50] [549/1000] d_loss: 0

[0/50] [666/1000] d_loss: 0.3632, g_loss: 3.556
(16, 1, 1, 105)
[0/50] [667/1000] d_loss: 0.2032, g_loss: 3.452
(16, 1, 1, 105)
[0/50] [668/1000] d_loss: 0.2103, g_loss: 3.582
(16, 1, 1, 105)
[0/50] [669/1000] d_loss: 0.2319, g_loss: 4.077
(16, 1, 1, 105)
[0/50] [670/1000] d_loss: 0.2017, g_loss: 3.251
(16, 1, 1, 105)
[0/50] [671/1000] d_loss: 0.1896, g_loss: 5.084
(16, 1, 1, 105)
[0/50] [672/1000] d_loss: 0.2237, g_loss: 3.79
(16, 1, 1, 105)
[0/50] [673/1000] d_loss: 0.2341, g_loss: 3.962
(16, 1, 1, 105)
[0/50] [674/1000] d_loss: 0.2037, g_loss: 4.232
(16, 1, 1, 105)
[0/50] [675/1000] d_loss: 0.2373, g_loss: 3.738
(16, 1, 1, 105)
[0/50] [676/1000] d_loss: 0.197, g_loss: 4.183
(16, 1, 1, 105)
[0/50] [677/1000] d_loss: 0.3295, g_loss: 2.616
(16, 1, 1, 105)
[0/50] [678/1000] d_loss: 0.1219, g_loss: 4.734
(16, 1, 1, 105)
[0/50] [679/1000] d_loss: 0.08787, g_loss: 2.936
(16, 1, 1, 105)
[0/50] [680/1000] d_loss: 0.303, g_loss: 3.565
(16, 1, 1, 105)
[0/50] [681/1000] d_loss: 0.3782, g_loss: 

[0/50] [799/1000] d_loss: 1.7, g_loss: 2.725
(16, 1, 1, 105)
[0/50] [800/1000] d_loss: 0.5769, g_loss: 2.932
(16, 1, 1, 105)
[0/50] [801/1000] d_loss: 0.3367, g_loss: 3.76
(16, 1, 1, 105)
[0/50] [802/1000] d_loss: 0.4584, g_loss: 3.54
(16, 1, 1, 105)
[0/50] [803/1000] d_loss: 0.4232, g_loss: 4.356
(16, 1, 1, 105)
[0/50] [804/1000] d_loss: 0.5094, g_loss: 2.202
(16, 1, 1, 105)
[0/50] [805/1000] d_loss: 0.4004, g_loss: 2.816
(16, 1, 1, 105)
[0/50] [806/1000] d_loss: 0.3144, g_loss: 3.531
(16, 1, 1, 105)
[0/50] [807/1000] d_loss: 0.5362, g_loss: 4.47
(16, 1, 1, 105)
[0/50] [808/1000] d_loss: 0.3619, g_loss: 2.256
(16, 1, 1, 105)
[0/50] [809/1000] d_loss: 0.5626, g_loss: 2.84
(16, 1, 1, 105)
[0/50] [810/1000] d_loss: 0.3381, g_loss: 3.206
(16, 1, 1, 105)
[0/50] [811/1000] d_loss: 0.3597, g_loss: 3.362
(16, 1, 1, 105)
[0/50] [812/1000] d_loss: 0.4545, g_loss: 2.479
(16, 1, 1, 105)
[0/50] [813/1000] d_loss: 0.6431, g_loss: 3.308
(16, 1, 1, 105)
[0/50] [814/1000] d_loss: 0.4222, g_loss: 3.991

(16, 1, 1, 105)
[0/50] [931/1000] d_loss: 0.4672, g_loss: 3.019
(16, 1, 1, 105)
[0/50] [932/1000] d_loss: 0.3306, g_loss: 1.501
(16, 1, 1, 105)
[0/50] [933/1000] d_loss: 0.418, g_loss: 2.666
(16, 1, 1, 105)
[0/50] [934/1000] d_loss: 0.4485, g_loss: 2.052
(16, 1, 1, 105)
[0/50] [935/1000] d_loss: 0.3405, g_loss: 2.365
(16, 1, 1, 105)
[0/50] [936/1000] d_loss: 0.4949, g_loss: 3.861
(16, 1, 1, 105)
[0/50] [937/1000] d_loss: 0.3841, g_loss: 4.425
(16, 1, 1, 105)
[0/50] [938/1000] d_loss: 0.3287, g_loss: 3.799
(16, 1, 1, 105)
[0/50] [939/1000] d_loss: 0.4442, g_loss: 3.173
(16, 1, 1, 105)
[0/50] [940/1000] d_loss: 0.4275, g_loss: 2.925
(16, 1, 1, 105)
[0/50] [941/1000] d_loss: 0.299, g_loss: 1.637
(16, 1, 1, 105)
[0/50] [942/1000] d_loss: 0.4474, g_loss: 3.966
(16, 1, 1, 105)
[0/50] [943/1000] d_loss: 0.864, g_loss: 0.4047
(16, 1, 1, 105)
[0/50] [944/1000] d_loss: 1.972, g_loss: 3.498
(16, 1, 1, 105)
[0/50] [945/1000] d_loss: 0.4002, g_loss: 2.913
(16, 1, 1, 105)
[0/50] [946/1000] d_loss: 0

ValueError: Error when checking input: expected dense_57_input to have 2 dimensions, but got array with shape (100, 1, 1, 5)